In [12]:
import pandas as pd
import numpy as np
import os

# Set options to display all rows and columns without truncation.
pd.options.display.max_rows = None
pd.options.display.max_columns = None

MAIN_DATA_FILE = 'C:/Users/511232/Desktop/DSS/MERGING GOOGLESHEETS QUESTIONNAIRES/codes/masterfile.xlsx'

In [13]:
df=pd.read_excel(MAIN_DATA_FILE)
df.columns

Index(['الفصل', 'المؤشر', 'الدولة', 'السنة', 'العدد', 'التوفر الكلي',
       'التوفر حسب المواطنية', 'التوفر حسب المنطقة',
       'توفر نقطة البيانات (كلي)', 'توفر نقطة البيانات (المواطنة)',
       'توفر نقطة البيانات (المنطقة)'],
      dtype='object')

In [14]:
df.drop(columns=['التوفر الكلي', 'التوفر حسب المنطقة', 'التوفر حسب المواطنية',  'توفر نقطة البيانات (المواطنة)',
       'توفر نقطة البيانات (المنطقة)' ], inplace=True)
df.head()

,الفصل,المؤشر,الدولة,السنة,العدد,توفر نقطة البيانات (كلي)
0,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2010,NaN,0
1,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2011,NaN,0
2,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2012,NaN,0
3,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2013,NaN,0
4,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2014,NaN,0


In [15]:
def assign_bin_hardcoded(year):
    """Manually assigns a year to a specific, hardcoded bin."""
    if 2010 <= year < 2015:
        return '[2010-2015)'
    elif 2015 <= year < 2020:
        return '[2015-2020)'
    elif 2020 <= year <= 2025:  # NOTE: Includes 2025 as requested
        return '[2020-2025]'
    else:
        return np.nan


print("Creating year bins with hardcoded ranges...")
df['year_bins'] = df['السنة'].apply(assign_bin_hardcoded)

bin_labels = ['[2010-2015)', '[2015-2020)', '[2020-2025]']
choices = [1, 2, 3]

# Create a list of boolean conditions by comparing the column to each label
conditions = [df['year_bins'] == label for label in bin_labels]
df['year_bin_code'] = np.select(conditions, choices, default=0)

#adding jitter on توفر نقطة البيانات (كلي)
# Define the original and new column names
original_col = 'توفر نقطة البيانات (كلي)'
new_col = 'jittered_availability' # You can name this whatever you like

# Create the new column as a copy of the original
df[new_col] = df[original_col].astype(float) # Copy and convert to float in one step

# Create a boolean mask from the ORIGINAL column to identify rows where the value is 1
mask = df[original_col] == 1

# Get the number of '1's that need jitter
num_to_jitter = mask.sum()

# Generate a small random number for each '1'
jitter = np.random.uniform(-0.5, 0.5, num_to_jitter)

# Apply the jitter to the NEW column where the mask is True
df.loc[mask, new_col] += jitter

df.head()

Creating year bins with hardcoded ranges...


,الفصل,المؤشر,الدولة,السنة,العدد,توفر نقطة البيانات (كلي),year_bins,year_bin_code,jittered_availability
0,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2010,NaN,0,[2010-2015),1,0.0
1,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2011,NaN,0,[2010-2015),1,0.0
2,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2012,NaN,0,[2010-2015),1,0.0
3,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2013,NaN,0,[2010-2015),1,0.0
4,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2014,NaN,0,[2010-2015),1,0.0


In [16]:
# Define the column names
group_col = ['الفصل','الدولة','year_bins']
value_col = 'توفر نقطة البيانات (كلي)'
new_col = 'availability_%'

# Calculate the percentage within each group and broadcast the result back
df[new_col] = df.groupby(group_col)[value_col].transform(
    lambda x: (x.sum() / x.size) * 100
)

df.head()

,الفصل,المؤشر,الدولة,السنة,العدد,توفر نقطة البيانات (كلي),year_bins,year_bin_code,jittered_availability,availability_%
0,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2010,NaN,0,[2010-2015),1,0.0,28.571429
1,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2011,NaN,0,[2010-2015),1,0.0,28.571429
2,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2012,NaN,0,[2010-2015),1,0.0,28.571429
3,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2013,NaN,0,[2010-2015),1,0.0,28.571429
4,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2014,NaN,0,[2010-2015),1,0.0,28.571429


In [17]:
aggregated_df = df.groupby(['الفصل','المؤشر','الدولة','year_bins']).agg(
    availability_pct=('availability_%', 'first'),
    year_bin_code=('year_bin_code', 'first')
).reset_index()

output_filename = 'availability_by_lastbin.xlsx'
aggregated_df.to_excel(output_filename, index=False)

In [ ]:
df.to_excel('C:/Users/511232/Desktop/DSS/MERGING GOOGLESHEETS QUESTIONNAIRES/codes/availability_with_jitter.xlsx', index=False)

In [ ]:
df.head()

,الفصل,المؤشر,الدولة,السنة,العدد,توفر نقطة البيانات (كلي),year_bins,year_bin_code,jittered_availability,availability_%
0,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2010,NaN,0,[2010-2015),1,0.0,38.598697
1,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2011,NaN,0,[2010-2015),1,0.0,38.598697
2,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2012,NaN,0,[2010-2015),1,0.0,38.598697
3,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2013,NaN,0,[2010-2015),1,0.0,38.598697
4,Housing,الوحدات السكنية الماهولة حسب المصدر الرئيسي لم...,الأردن,2014,NaN,0,[2010-2015),1,0.0,38.598697
